<a href="https://colab.research.google.com/github/shreekar2005/Cheminformatics_DC/blob/main/features_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading dataset

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("jessicali9530/lfw-dataset")
print(path)

100%|██████████| 112M/112M [00:00<00:00, 143MB/s]

Extracting files...


/root/.cache/kagglehub/datasets/jessicali9530/lfw-dataset/versions/4


In [ ]:
import os
images_address="/root/.cache/kagglehub/datasets/jessicali9530/lfw-dataset/versions/4/lfw-deepfunneled/lfw-deepfunneled"
labels=os.listdir(images_address)
paths_of_images=[]
for root, dirs, files in os.walk(images_address):
    for file in files:
        paths_of_images.append(os.path.join(root, file))

print(paths_of_images[:5])

['/root/.cache/kagglehub/datasets/jessicali9530/lfw-dataset/versions/4/lfw-deepfunneled/lfw-deepfunneled/Jason_Sorens/Jason_Sorens_0001.jpg', '/root/.cache/kagglehub/datasets/jessicali9530/lfw-dataset/versions/4/lfw-deepfunneled/lfw-deepfunneled/Guus_Hiddink/Guus_Hiddink_0001.jpg', '/root/.cache/kagglehub/datasets/jessicali9530/lfw-dataset/versions/4/lfw-deepfunneled/lfw-deepfunneled/Gerrit_Zalm/Gerrit_Zalm_0001.jpg', '/root/.cache/kagglehub/datasets/jessicali9530/lfw-dataset/versions/4/lfw-deepfunneled/lfw-deepfunneled/Kenenisa_Bekele/Kenenisa_Bekele_0001.jpg', '/root/.cache/kagglehub/datasets/jessicali9530/lfw-dataset/versions/4/lfw-deepfunneled/lfw-deepfunneled/Jeremy_Shockey/Jeremy_Shockey_0001.jpg']


# Filtering dataset

filtering persons who has less than 10 images and storing them in form of dictionary.

person_imagepath = {person_name : [img_path1, img_path2, img_path3, ...]}

In [ ]:
import re
i=0
person_imagepath={}
for path in paths_of_images:
    i+=1
    matchpattern = re.search(r'/([^/]+)_\d+\.jpg$', path)
    if matchpattern:
        if matchpattern.group(1) not in person_imagepath:
            person_imagepath[matchpattern.group(1)]=[]
        person_imagepath[matchpattern.group(1)].append(path)

person_imagepath = {person: images for person, images in person_imagepath.items() if len(images) >= 10}
total_filtered_datapoints=0
for person in person_imagepath:
    total_filtered_datapoints+=len(person_imagepath[person])

print("total datapoints having person atleast 10 images are",total_filtered_datapoints)


total datapoints having person atleast 10 images are 4324


# Extracting CNN features


Using pretrained CNN (ResNet)

In [ ]:
#This code can be used to extract pretrained CNN (ResNet) Features.
#It takes Image Path and returns 2048-dimensional feature.
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image

# Load pre-trained ResNet-50 model
resnet = models.resnet50(pretrained=True)
# Remove the last fully connected layer
resnet = nn.Sequential(*list(resnet.children())[:-1])
# Set the model to evaluation mode
resnet.eval()

# Define a function to extract features from an image
def extract_features(image_path, model):
    # Load and preprocess the image
    image = Image.open(image_path).convert('RGB')
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    image = preprocess(image)
    # Add batch dimension
    image = image.unsqueeze(0)
    # Extract features
    with torch.no_grad():
        features = model(image)
    # Remove the batch dimension
    features = features.squeeze(0)
    return features

In [ ]:
# saving featrues
import pandas as pd
import numpy as np
data_list=[]
i=0
for person in person_imagepath :
  for path in person_imagepath[person]:
      i+=1
      features = extract_features(path, resnet)
      features = features.flatten().numpy()
      datapoint = np.append(features, person)
      data_list.append(datapoint)
      if i%100==0:
          print(i,"/",total_filtered_datapoints,"converted")
df = pd.DataFrame(data_list)
df.to_csv("filtered_CNN_features_dataset.csv")
df

# Extracting HoG features

#Extracting LBP features